In [10]:
import json 
import numpy as np

def get_number_regions(experiment): 
  number_regions = ! less $experiment/regions.bed.gz | wc -l
  number_regions, = number_regions
  return int(number_regions)

def get_confusion_counts_core(experiment, json_filename): 
  with open('{}/{}'.format(experiment, json_filename)) as json_file:
    return json.load(json_file)

def get_confusion_counts(experiment): 
  try: 
    return get_confusion_counts_core(experiment, 'summary.txt')
  except FileNotFoundError: 
    pass 

  return get_confusion_counts_core(experiment, 'counts.json')

def get_eventCount_callCount(experiment): 
  try: 
    counts = get_confusion_counts(experiment)
    try: 
      assert counts['TP-base'] == counts['TP-call']    
    except KeyError: 
      pass 
    event_count = counts['TP-base'] + counts['FN']
    call_count = counts['TP-base'] + counts['FP']
    return event_count, call_count
  except FileNotFoundError: 
    return None, None

def get_TPR_FDR(experiment):
  try: 
    counts = get_confusion_counts(experiment)
    event_count, call_count = get_eventCount_callCount(experiment)
    TPR = counts['TP-base']/float(event_count)
    FDR = counts['FP']/float(call_count) 
    return TPR, FDR
  except (FileNotFoundError, ZeroDivisionError): 
    return None, None

def get_TP_eventCount(experiment):
  try: 
    counts = get_confusion_counts(experiment)
    event_count, _ = get_eventCount_callCount(experiment)
    return counts['TP-base'], event_count
  except FileNotFoundError: 
    return None, None

def get_FP_callCount(experiment):
  try: 
    counts = get_confusion_counts(experiment)
    _, call_count = get_eventCount_callCount(experiment)
    return counts['FP'], call_count
  except FileNotFoundError: 
    return None, None

def get_TPRs_FDRs(experiments): 
  TPRs, FDRs = zip(*[get_TPR_FDR(experiment) for experiment in experiments])  
  return TPRs, FDRs

def get_numbers_regions(experiments): 
  return [get_number_regions(experiment) for experiment in experiments]

def get_eventCounts_callCounts(experiments): 
  event_counts, call_counts = zip(*[get_eventCount_callCount(experiment) for experiment in experiments])
  return event_counts, call_counts

def get_experiments(consortium, population, sample, svtype, caller): 
  experiments = ! ls -d $consortium/data/$population,$sample/truvari-$svtype-*,*-pacbio-$caller
  return experiments

def get_experiments_and_size_ranges(consortium, population, sample, svtype, caller):
  experiments = get_experiments(consortium, population, sample, svtype, caller)
  region_size_ranges_str = [experiment.split('-')[2] for experiment in experiments]
  region_size_ranges = [region_size_range_str.split(',') for region_size_range_str in region_size_ranges_str]
  region_size_ranges = [(int(start), int(end)) for start, end in region_size_ranges]
  return experiments, region_size_ranges_str, region_size_ranges

def zip_and_sort(experiments_and_size_ranges):
  l = list(zip(*experiments_and_size_ranges))    
  l.sort(key=lambda x: x[2])
  return tuple(zip(*l))[:2]

zip_and_sort(get_experiments_and_size_ranges('chaisson_2019', 'PUR', 'HG00733', 'DEL', 'manta'))

(('chaisson_2019/data/PUR,HG00733/truvari-DEL-600,625-pacbio-manta',
  'chaisson_2019/data/PUR,HG00733/truvari-DEL-625,650-pacbio-manta',
  'chaisson_2019/data/PUR,HG00733/truvari-DEL-650,700-pacbio-manta',
  'chaisson_2019/data/PUR,HG00733/truvari-DEL-700,800-pacbio-manta',
  'chaisson_2019/data/PUR,HG00733/truvari-DEL-800,1000-pacbio-manta',
  'chaisson_2019/data/PUR,HG00733/truvari-DEL-1000,2000-pacbio-manta',
  'chaisson_2019/data/PUR,HG00733/truvari-DEL-2000,100000-pacbio-manta'),
 ('600,625',
  '625,650',
  '650,700',
  '700,800',
  '800,1000',
  '1000,2000',
  '2000,100000'))

In [11]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def get_font(size=20, family='Arial'): 
  return dict(family=family, color='black', size=size)

def get_marker(color=None, linewidth=1, linecolor='black'): 
  return dict(
    line=dict(width=linewidth, color=linecolor),
    color=color
  )

def update_axes(fig, axis, text=None, showline=True, ticks='outside', linewidth=1.5, range_=None, row=1, col=1): 
  update = {
    'x': fig.update_xaxes,
    'y': fig.update_yaxes
  }
  
  update[axis]( 
    title=dict(text=text),
    showgrid=False,
    showline=showline, 
    linewidth=linewidth, 
    linecolor='black',
    ticks=ticks, 
    tickwidth=linewidth, 
    tickcolor='black', 
    ticklen=10,
    range=range_,
    row=row,
    col=col
  )

def get_slop(consortium, population, sample): 
  with open(f'{consortium}/data/{population},{sample}/config.json') as json_file:
    return int(json.load(json_file)['makeRegions']['slop'])

def region_to_VNTR(consortium, population, sample, region_size_ranges_str): 
  region_size_ranges = [region_size_range_str.split(',') for region_size_range_str in region_size_ranges_str]
  return [f'{int(start)-2*get_slop(consortium, population, sample)}-{int(end)-2*get_slop(consortium, population, sample)}' 
          for start, end in region_size_ranges]

def get_VNTR_size_ranges__TPRs__FDRs(consortium, population, sample, svtype, caller): 
  experiments, region_size_ranges_str = zip_and_sort(get_experiments_and_size_ranges(consortium, population, sample, svtype, caller))
  TPRs, FDRs = get_TPRs_FDRs(experiments)
  VNTR_size_ranges = region_to_VNTR(consortium, population, sample, region_size_ranges_str)
  return VNTR_size_ranges, TPRs, FDRs

def get_VNTR_size_ranges__numbers_regions__event_counts__call_counts(consortium, population, sample, svtype, caller): 
  experiments, region_size_ranges_str = zip_and_sort(get_experiments_and_size_ranges(consortium, population, sample, svtype, caller))
  numbers_regions = get_numbers_regions(experiments)
  event_counts, call_counts = get_eventCounts_callCounts(experiments)
  VNTR_size_ranges = region_to_VNTR(consortium, population, sample, region_size_ranges_str)
  return VNTR_size_ranges, numbers_regions, event_counts, call_counts

def create_bar_chart_AB(consortium, population, sample, svtype, caller, chart_type, color, row, col): 
  VNTR_size_ranges, TPRs, FDRs = get_VNTR_size_ranges__TPRs__FDRs(consortium, population, sample, svtype, caller)
  y = { 
    'TPR': TPRs, 
    'FDR': FDRs,
  }
  name = { 
    'manta': 'manta',
    'trfermikit.unitigSupport.thinned': 'trfermikit',
    'combined': 'trfermikit + manta'
  }
      
  def get_showlegend():
    if row == 1 and col == 1: return True 
    else: return False  

  return go.Bar(
    x=VNTR_size_ranges, 
    y=y[chart_type], 
    name=name[caller],
    marker=get_marker(color=color),
    showlegend=get_showlegend()
  )

In [12]:
def plot_performance_AB(consortium, population, sample, svtype, chart_type, fig, row, col): 
  fig.add_trace(
    create_bar_chart_AB(consortium, population, sample, svtype, 'trfermikit.unitigSupport.thinned', chart_type, color='red', row=row, col=col),
    row=row, 
    col=col
  ) 
  fig.add_trace(
    create_bar_chart_AB(consortium, population, sample, svtype, 'manta', chart_type, color='blue', row=row, col=col),
    row=row,
    col=col
  ) 
  fig.add_trace(
    create_bar_chart_AB(consortium, population, sample, svtype, 'combined', chart_type, color='yellow', row=row, col=col),
    row=row,
    col=col
  ) 
  update_axes(fig, 'x', 'VNTR length (bps)', showline=False, ticks=None, row=row, col=col) 
  y_text = { 
    'TPR': 'Recall, TP / #events',
    'FDR': 'False Discovery Rate, <br>FP / #calls'    
  }  
  update_axes(fig, 'y' , y_text[chart_type], range_=[0, 1], row=row, col=col) 

def plot_counts(consortium, population, sample, svtype, fig, row, col, range_=None): 
  caller = 'trfermikit.unitigSupport.thinned'

  caller_legend = 'trfermikit' if caller == 'trfermikit.unitigSupport.thinned' else 'manta'

  VNTR_size_ranges, numbers_regions, event_counts, call_counts = \
    get_VNTR_size_ranges__numbers_regions__event_counts__call_counts(consortium, population, sample, svtype, caller=caller)

  def get_showlegend():
    if row == 1 and col == 3: return True 
    elif row == 2 and col == 1: return True 
    else: return False  

  fig.add_trace(
    go.Bar(
      x=VNTR_size_ranges,
      y=numbers_regions,
      name='VNTRs',
      marker=get_marker(color='grey'),
      showlegend=get_showlegend()
    ),
    row=row, 
    col=col
  ) 
  fig.add_trace(
    go.Bar(
      x=VNTR_size_ranges,
      y=event_counts,
      name='real events',
      marker=get_marker(color='cyan'),
      showlegend=get_showlegend()
    ), 
    row=row,
    col=col
  ) 
  fig.add_trace(
    go.Bar(
      x=VNTR_size_ranges, 
      y=call_counts, 
      name=f'{caller_legend} calls',
      marker=get_marker(color='magenta'),
      showlegend=get_showlegend()
    ), 
    row=row,
    col=col
  )
  update_axes(fig, 'x', 'VNTR length (bps)', showline=False, ticks=None, row=row, col=col)
  update_axes(fig, 'y' , 'count', range_=range_, row=row, col=col)
  fig.update_yaxes(type='log', row=row, col=col)


In [13]:
def check(manta_callset): 
  if manta_callset != 'fn' and manta_callset != 'tp-base': 
    raise Exception
  else: 
    pass

def list_to_value(list_):
  try: 
    value, = list_
    return int(value)
  except ValueError: 
    return None
      
def tr_fermikit_TP_intersect_manta(manta_callset, manta_experiment, trfermikit_experiment, overlap_fraction, root): 
  list_ = ! $root/bin/bedtools intersect -u -wb -f $overlap_fraction -r \
    -a $manta_experiment/$manta_callset".vcf" \
    -b $trfermikit_experiment/tp-base.vcf \
    | wc -l 
  return list_to_value(list_)

def manta_less_tr_fermikit_TP(manta_callset, manta_experiment, trfermikit_experiment, overlap_fraction, root):
  list_ = ! $root/bin/bedtools subtract -A -f $overlap_fraction -r \
    -a $manta_experiment/$manta_callset".vcf" \
    -b $trfermikit_experiment/tp-base.vcf \
  | wc -l 
  return list_to_value(list_)

def get_manta_counts(manta_callset, manta_experiment, trfermikit_experiment): 
  overlap_fraction = 0.9
  root = "/scratch/ucgd/lustre-work/quinlan/u6018199/chaisson_2019/analysis/locally_assemble_short_reads/trfermikit"
  overlap_count = tr_fermikit_TP_intersect_manta(manta_callset, manta_experiment, trfermikit_experiment, overlap_fraction, root)
  manta_exclusive_count = manta_less_tr_fermikit_TP(manta_callset, manta_experiment, trfermikit_experiment, overlap_fraction, root)
  return overlap_count, manta_exclusive_count
  
def get_manta_fraction(manta_callset, manta_experiment, trfermikit_experiment):
  check(manta_callset)
  overlap_count, manta_exclusive_count = get_manta_counts(manta_callset, manta_experiment, trfermikit_experiment)    
  try: 
    return overlap_count/float(overlap_count + manta_exclusive_count)
  except TypeError: 
    return None

def plot_fraction_of_manta_FNs_recovered_by_trfermikit(consortium, population, sample, svtype, fig, row, col, range_=[0, None]): 
  manta_experiments, region_size_ranges_str = zip_and_sort(get_experiments_and_size_ranges(consortium, population, sample, svtype, caller='manta'))
  trfermikit_experiments, region_size_ranges_str = zip_and_sort(get_experiments_and_size_ranges(consortium, population, sample, svtype, caller='trfermikit.unitigSupport.thinned'))

  fractions_of_manta_FN = [
    get_manta_fraction(
      manta_callset='fn', 
      manta_experiment=manta_experiment, 
      trfermikit_experiment=trfermikit_experiment
    ) 
    for manta_experiment, trfermikit_experiment in zip(manta_experiments, trfermikit_experiments)
  ]

  VNTR_size_ranges = region_to_VNTR(consortium, population, sample, region_size_ranges_str)

  fig.add_trace(
    go.Bar(
      x=VNTR_size_ranges,
      y=fractions_of_manta_FN,
      marker=get_marker(color='black'),
      showlegend=False
    ),
    row=row, 
    col=col
  ) 
  update_axes(fig, 'x', 'VNTR length (bps)', showline=False, ticks=None, row=row, col=col)
  update_axes(fig, 'y' , 'fraction of manta FNs <br>recovered by trfermikit', range_=range_, row=row, col=col)

def plot_Fig2(consortium, population, sample, svtype): 
  x_title = 'VNTR length (bps)'

  fig = make_subplots(
    rows=2,
    cols=2,
    column_widths=[1, 1],
    row_heights=[1, 1],
    specs=2*[2*[{"type": "bar"}]],
    horizontal_spacing=0.2,
    vertical_spacing=0.01,
    shared_xaxes=True,
    x_title=x_title
  ) 
  
  plot_performance_AB(consortium, population, sample, svtype, chart_type='TPR', fig=fig, row=1, col=1)
  plot_performance_AB(consortium, population, sample, svtype, chart_type='FDR', fig=fig, row=1, col=2)
  plot_counts(consortium, population, sample, svtype, fig=fig, row=2, col=1)
  plot_fraction_of_manta_FNs_recovered_by_trfermikit(consortium, population, sample, svtype, fig=fig, row=2, col=2)

  fig.update_layout(dict(
      xaxis=dict(type='category'),
      plot_bgcolor='white',
      font=get_font(),
      width=1000,
      height=800,
      margin=dict(
        l=50,
        r=50,
        b=175,
        t=50,
        pad=0
      )
  ))

  fig.update_xaxes(
    title=dict(text=None),
    tickangle=45
  )

  fig.update_annotations(font=get_font(size=26))

  annotations = fig.layout.annotations
  annotation_texts = [annotation['text'] for annotation in annotations]
  annotations[annotation_texts.index(x_title)].update(y=-0.15)

  fig.write_image('Fig2.pdf')

  fig.show()

plot_Fig2('chaisson_2019', 'CHS', 'HG00514', 'DEL')

In [14]:
def get_overall_TPR_FDR_core(experiments): 
  TPs, event_counts = zip(*[get_TP_eventCount(experiment) for experiment in experiments])  
  try: 
    TPR = np.sum(TPs)/np.sum(event_counts)
  except TypeError: 
    TPR = None 
  FPs, call_counts = zip(*[get_FP_callCount(experiment) for experiment in experiments])  
  try: 
    FDR = np.sum(FPs)/np.sum(call_counts)
  except TypeError: 
    FDR = None
  return TPR, FDR
  
def get_overall_TPR_FDR(consortium, population, sample, svtype, caller): 
  experiments = get_experiments(consortium, population, sample, svtype, caller)
  TPR, FDR = get_overall_TPR_FDR_core(experiments)
  return TPR, FDR 

get_overall_TPR_FDR('HGSVC2', 'ESN', 'HG03125', 'DEL', 'trfermikit.unitigSupport.thinned')

(0.3884422110552764, 0.40400925212027755)

In [15]:
def get_manta_counts_wrapper(manta_callset, manta_experiment, trfermikit_experiment):
  check(manta_callset)
  overlap_count, manta_exclusive_count = get_manta_counts(manta_callset, manta_experiment, trfermikit_experiment)    
  return overlap_count, manta_exclusive_count

def get_overall_manta_fraction(consortium, population, sample, svtype): 
  manta_experiments, _ = zip_and_sort(get_experiments_and_size_ranges(consortium, population, sample, svtype, caller='manta'))
  trfermikit_experiments, _ = zip_and_sort(get_experiments_and_size_ranges(consortium, population, sample, svtype, caller='trfermikit.unitigSupport.thinned'))

  manta_counts = [
    get_manta_counts_wrapper(
      manta_callset='fn', 
      manta_experiment=manta_experiment, 
      trfermikit_experiment=trfermikit_experiment
    ) 
    for manta_experiment, trfermikit_experiment in zip(manta_experiments, trfermikit_experiments)
  ]

  overlap_counts, manta_exclusive_counts = zip(*manta_counts)
  return np.sum(overlap_counts)/(np.sum(overlap_counts) + np.sum(manta_exclusive_counts))

get_overall_manta_fraction('chaisson_2019', 'CHS', 'HG00514', 'DEL')

0.1737866899157222

In [16]:
import os 

def get_consortia_populations_samples_core(consortium): 
  populations_samples = ! ls -d $consortium/data/*
  return [[consortium] + os.path.basename(os.path.normpath(population_sample)).split(',') for population_sample in populations_samples]

def get_consortia_populations_samples(): 
  return get_consortia_populations_samples_core('chaisson_2019') + get_consortia_populations_samples_core('HGSVC2')

get_consortia_populations_samples() 

[['chaisson_2019', 'CHS', 'HG00514'],
 ['chaisson_2019', 'PUR', 'HG00733'],
 ['chaisson_2019', 'YRI', 'NA19240'],
 ['HGSVC2', 'CEU', 'NA12878'],
 ['HGSVC2', 'ESN', 'HG03125'],
 ['HGSVC2', 'GWD', 'HG02818'],
 ['HGSVC2', 'MSL', 'HG03486']]

In [17]:
def create_bar_chart_AB_all_samples(svtype, caller, chart_type, color):  
  consortia_populations_samples = get_consortia_populations_samples()
  consortia, populations, samples = zip(*consortia_populations_samples) 
  
  populations_samples = [f'{population},{sample}' for population, sample in zip(populations, samples)]

  overall_TPRs, overall_FDRs = zip(*[
    get_overall_TPR_FDR(consortium, population, sample, svtype, caller) 
    for consortium, population, sample in consortia_populations_samples
  ])

  y = { 
    'TPR': overall_TPRs, 
    'FDR': overall_FDRs,
  }
  name = { 
    'manta': 'manta',
    'trfermikit.unitigSupport.thinned': 'trfermikit',
    'combined': 'trfermikit + manta'
  }
  def get_showlegend(): 
    if chart_type == 'TPR': return True
    elif chart_type == 'FDR': return False 
    else: raise ValueError
    
  return go.Bar(
    x=populations_samples, 
    y=y[chart_type], 
    name=name[caller],
    marker=get_marker(color=color),
    showlegend=get_showlegend()
  )
  
def plot_performance_AB_all_samples(svtype, chart_type, fig, row, col): 
  fig.add_trace(
    create_bar_chart_AB_all_samples(svtype, 'trfermikit.unitigSupport.thinned', chart_type, color='red'),
    row=row, 
    col=col
  ) 
  fig.add_trace(
    create_bar_chart_AB_all_samples(svtype, 'manta', chart_type, color='blue'),
    row=row,
    col=col
  ) 
  fig.add_trace(
    create_bar_chart_AB_all_samples(svtype, 'combined', chart_type, color='yellow'),
    row=row,
    col=col
  ) 
  update_axes(fig, 'x', None, showline=False, ticks=None, row=row, col=col) 
  y_text = { 
    'TPR': 'Recall, TP / #events',
    'FDR': 'False Discovery Rate, FP / #calls'    
  }  
  update_axes(fig, 'y' , y_text[chart_type], range_=[0, 1], row=row, col=col) 

def create_bar_chart_overall_manta_fraction_all_samples(svtype):  
  consortia_populations_samples = get_consortia_populations_samples()
  consortia, populations, samples = zip(*consortia_populations_samples) 
  
  populations_samples = [f'{population},{sample}' for population, sample in zip(populations, samples)]

  overall_manta_fractions = [
    get_overall_manta_fraction(consortium, population, sample, svtype)
    for consortium, population, sample in consortia_populations_samples
  ]

  return go.Bar(
    x=populations_samples, 
    y=overall_manta_fractions, 
    marker=get_marker(color='black'),
    showlegend=False
  )
 
def plot_overall_manta_fraction_all_samples(svtype, fig, row, col): 
  fig.add_trace(
    create_bar_chart_overall_manta_fraction_all_samples(svtype),
    row=row, 
    col=col
  ) 
  update_axes(fig, 'x', None, showline=False, ticks=None, row=row, col=col) 
  update_axes(fig, 'y' , 'fraction of manta FNs <br>recovered by trfermikit', range_=[0, 1], row=row, col=col) 

def plot_Fig3(svtype): 
  x_title = 'sample'

  fig = make_subplots(
    rows=1, 
    cols=3,
    column_widths=[1, 1, 1],
    row_heights=[1],
    specs=[[{"type": "bar"}, {"type": "bar"}, {"type": "bar"}]],
    horizontal_spacing=0.125,
    x_title=x_title
  ) 
  
  plot_performance_AB_all_samples(svtype, chart_type='TPR', fig=fig, row=1, col=1)
  plot_performance_AB_all_samples(svtype, chart_type='FDR', fig=fig, row=1, col=2)
  plot_overall_manta_fraction_all_samples(svtype, fig=fig, row=1, col=3)

  fig.update_layout(dict(
      xaxis=dict(type='category'),
      plot_bgcolor='white',
      font=get_font(),
      width=1500,
      height=600,
      margin=dict(
        l=50,
        r=50,
        b=200,
        t=50,
        pad=0
      )
  ))

  fig.update_xaxes(tickangle=45)

  fig.update_annotations(font=get_font(size=26))

  annotations = fig.layout.annotations
  annotation_texts = [annotation['text'] for annotation in annotations]
  annotations[annotation_texts.index(x_title)].update(y=-0.3)

  fig.write_image('Fig3.pdf')

  fig.show()

plot_Fig3('DEL') 

In [18]:
def plot_FigS3(svtype): 
  consortia_populations_samples = get_consortia_populations_samples()

  number_samples = len(consortia_populations_samples)

  consortia, populations, samples = zip(*consortia_populations_samples)   
  populations_samples = [f'{population},{sample}' for population, sample in zip(populations, samples)]

  x_title = 'VNTR length (bps)'

  fig = make_subplots(
    rows=number_samples,
    cols=4,
    row_heights=number_samples*[1],
    column_widths=4*[1],
    specs=number_samples*[4*[{"type": "bar"}]],
    # horizontal_spacing=0.1,
    vertical_spacing=0.01,
    row_titles=populations_samples,
    column_titles=['Recall, TP / #events', 'FDR, FP / #calls', 'Count', 'fraction of manta FNs <br>recovered by trfermikit'],
    shared_xaxes=True,
    x_title=x_title
  ) 
  
  for index, (consortium, population, sample) in enumerate(consortia_populations_samples): 
    row = index + 1
    plot_performance_AB(consortium, population, sample, svtype, chart_type='TPR', fig=fig, row=row, col=1)
    plot_performance_AB(consortium, population, sample, svtype, chart_type='FDR', fig=fig, row=row, col=2)
    plot_counts(consortium, population, sample, svtype, fig=fig, row=row, col=3, range_=[np.log10(100), np.log10(100000)])
    plot_fraction_of_manta_FNs_recovered_by_trfermikit(consortium, population, sample, svtype, fig=fig, row=row, col=4, range_=[0, 1])

  fig.update_layout(dict(
      xaxis=dict(type='category'),
      plot_bgcolor='white',
      font=get_font(),
      width=1800,
      height=2000,
      margin=dict(
        l=50,
        r=50,
        b=150,
        t=100,
        pad=0
      )
  ))

  fig.update_xaxes(
    title=dict(text=None),
    tickangle=45
  )

  fig.update_yaxes(
    title=dict(text=None)
  )

  fig.update_annotations(font=get_font(size=26))

  annotations = fig.layout.annotations
  annotation_texts = [annotation['text'] for annotation in annotations]
  annotations[annotation_texts.index(x_title)].update(y=-0.04)

  fig.write_image('FigS3.pdf')

  fig.show()

plot_FigS3('DEL')